<a href="https://colab.research.google.com/github/Anandsure/Xcelerrator/blob/master/use_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [2]:
!pip install sklearn

In [0]:
from nltk.corpus import treebank
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [0]:
import pandas as pd

In [0]:
import random as r
import itertools 

In [6]:
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [0]:
sen = treebank.tagged_sents(tagset='universal')

In [0]:
# i am going to use a train, validation and a test set
train_test_cutoff = int(.80 * len(sen)) 
training_sentences = sen[:train_test_cutoff]
testing_sentences = sen[train_test_cutoff:]
train_val_cutoff = int(.25 * len(training_sentences))
validation_sentences = training_sentences[:train_val_cutoff]
training_sentences = training_sentences[train_val_cutoff:]

In [0]:
def add_basic_features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

In [0]:
def untag(tagged_sentence):
    return [w for w, _ in tagged_sentence]
def transform_to_dataset(tagged_sentences):
  X, y = [], []
  for pos_tags in tagged_sentences:
    for index, (term, class_) in enumerate(pos_tags):
        # I am going to do the splitting here
        X.append(add_basic_features(untag(pos_tags), index))
        y.append(class_)
  return X,y

In [0]:
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

In [0]:
from sklearn.feature_extraction import DictVectorizer
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_train + X_test + X_val)
#dict features to vectors
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)
X_val = dict_vectorizer.transform(X_val)